In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_excel("MDS_Assignment1_winequality.xlsx")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
X = df.iloc[:, :-1].values
y = df.iloc[:, 11].values
X = sm.add_constant(X)

print(X.shape)
print(y.shape)

(1599, 12)
(1599,)


# Question (1.)
## (a) (10%) Show the results of regression analysis as follows.

In [4]:

model = sm.OLS(y, X).fit()
summary = model.summary()

estimates = model.params
std_errors = model.bse
t_statistics = model.tvalues
p_values = model.pvalues

results_df = pd.DataFrame({'Estimates': estimates, 'Std. Errors': std_errors, 't-value': t_statistics, 'p-value': p_values})
results_df.index = ['Intercept', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
results_df  

,Estimates,Std. Errors,t-value,p-value
Intercept,21.965208,21.194575,1.036360,3.001921e-01
fixed acidity,0.024991,0.025949,0.963083,3.356528e-01
volatile acidity,-1.083590,0.121101,-8.947802,9.872361e-19
citric acid,-0.182564,0.147176,-1.240445,2.149942e-01
residual sugar,0.016331,0.015002,1.088599,2.764960e-01
chlorides,-1.874225,0.419283,-4.470070,8.373953e-06
free sulfur dioxide,0.004361,0.002171,2.008635,4.474495e-02
total sulfur dioxide,-0.003265,0.000729,-4.479830,8.004610e-06
density,-17.881164,21.633100,-0.826565,4.086079e-01
pH,-0.413653,0.191597,-2.158971,3.100189e-02


In [5]:
r_squared = model.rsquared
adjusted_r_squared = model.rsquared_adj

print("R-squared (R²):", r_squared)
print("Adjusted R-squared (Adjusted R²):", adjusted_r_squared)

R-squared (R²): 0.36055170303868767
Adjusted R-squared (Adjusted R²): 0.35611948421917006


## (b) (5%) The fitting of the linear regression is a good idea? If yes, why? If no, why? What’s the possible reason of poor fitting?

### Ans: No, it's not a good idea. The reason for poor fitting when using a linear regression model may be that 1-degree linear combination cannot effectively explain the relationship between the dependent variable and the independent variables.

## (c) (5%) Based on the results, rank the independent variables by p-values and which one are statistically significant variables with p-values<0.01? (i.e. 重要變數挑選)

In [6]:
significance_df = results_df.sort_values(by ="p-value")
significance_df[significance_df["p-value"] < 0.01]


,Estimates,Std. Errors,t-value,p-value
alcohol,0.276198,0.026484,10.429014,1.123029e-24
volatile acidity,-1.083590,0.121101,-8.947802,9.872361e-19
sulphates,0.916334,0.114337,8.014297,2.127228e-15
total sulfur dioxide,-0.003265,0.000729,-4.479830,8.004610e-06
chlorides,-1.874225,0.419283,-4.470070,8.373953e-06


### Ans : 1. alcohol, 2. volatile acidity, 3. total sulfur dioxide, 4. chlorides.

## (d) (15%) Testify the underlying assumptions of regression (1) Normality, (2) Independence, and (3) Homogeneity of Variance with respect to residual.

In [7]:
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.stattools import durbin_watson
from scipy import stats

# (1.) 
residuals = model.resid

# qq = ProbPlot(residuals)
# qq.qqplot(line='s', alpha=0.7)
# plt.title("Residuals QQ Plot")
# plt.show()

shapiro_test = stats.shapiro(residuals)

print("Shapiro-Wilk p-value:", shapiro_test[1])

# (2.) Durbin-Watson test
dw_test_statistic  = durbin_watson(residuals)
print("Durbin-Waston test statistics:", dw_test_statistic)
# (3.) Levene test
statistic, p_value = stats.levene(residuals, residuals, center='median')
print("Levene test statistics:", statistic)
print("Levene test p-value:", p_value)



Shapiro-Wilk p-value: 1.956532713620618e-08
Durbin-Waston test statistics: 1.7571401888793017
Levene test statistics: 0.0
Levene test p-value: 1.0


### Ans: 
### (1.) To assess the normality of residuals, we can conduct a Shapiro-Wilk test. The null hypothesis in the Shapiro test assumes that the sample follows a normal distribution. Since the p-value of the Shapiro test is less than 0.05 (at a 5% significance level), we reject the null hypothesis, indicating that the residuals do not follow a normal distribution.
### (2.) To assess the independence of residuals, we can conduct a Durbin-Watson test. The null hypothesis in the Durbin-Watson test posits that there is no serial correlation (autocorrelation) among the residuals in a regression model. In simpler terms, it assumes that the errors are uncorrelated and independent of each other. With a Durbin-Watson test p-value of 1.757, which is fairly close to 2, we fail to reject the null hypothesis, indicating that the residuals are indeed independent."
### (3.) To assess the homogeneity of variance in relation to the residuals, we can perform a Levene test. The null hypothesis of the Levene test assumes that the variances of the two groups are equal. Since the p-value is close to 1, we can't reject the null hypothesis, indicating that the variances of the residuals are homogeneous.

# Question (2.)
## (1.) (10%) How to handle the raw dataset via data preprocessing?

In [31]:
df = pd.read_csv("MDS_Assignment1_OnlineRetail.csv")
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010/12/1 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010/12/1 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011/12/9 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011/12/9 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011/12/9 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011/12/9 12:50,4.15,12680.0,France


In [44]:
result_df = df.pivot_table(index='InvoiceNo', columns='StockCode', values='Quantity').fillna(0)
result_df

StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C581490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C581499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Ans : The raw data is a long format dataframe. To transform the dataframe into format for extracting association rules (which is wide format), we can use df.pivot_table(). Every row in the pivot dataframe represents an invoice with every column represents the purchased quantity of individual stock. After the transformation, the shape of dataframe will switch from (541909, 6) to (25900, 4070) where 25900 is the invoice numbers and 4070 is the number of stock type.

## (2.) (10%) Define and what’s the top 10 association rules? Show the support, confidence, and lift to each specific rule, respectively?

## (3.) (5%) Please provide/guess the “story” to interpret one of top-10 rules you are interested in (Third column in dataset shows item Description).

## (4.) (10%) Give a visualization graph of your association rules. 